# General

### TO DO

- 2710017_BD2  
- Compare the CSV files and why rejected
- customers.csv - nb 4999 in the CSV file what have they done with the duplicate?
- Check image and URL requests
- Other augmentation?
- Cloud plot


### Assignment

- 1. Data Exploration - Explore the data and show you understand its structure and relations
- 2. Data Validation - Check the quality of the data. Is it complete? Are there obvious errors?
- x. Data Preparation - Addresss the issues identified, supliment/augment the data, restructure
    - Select Data - which to use and which to exclude .. with reasoning
    - Clean Data - 'correct, impute, remove' ...
    - Construct Data - derive new attributes as needed/helpful
    - Integrate Data - new datasets, augment, other sources
    - Format Data - reformat as needed, eg string to numeric, dates, categorical  

- ?? data structure, size
- ?? data quality, missing etc
- ?? References .. from lecture: But he did say we should mention sources for code including the course materials

#### Data Completeness
 - Missing Data - Identify and resolve by removal or inferring. Imputation and visualisation, examining descriptive stats
- Noisy - Random errors, eg from faulty sensors, data transmission
- Duplicates - Identify and eliminate duplicates, redundancy. NB: Can occur after data integration
- Inconsistency - Data items don't align, eg DOB and age

# Analysis & Visualisation

## To Do


- Customers
    - by age bands eg under 25, 24 to 40 etc
    - sales counts and value
    - bar chart to visualise

- Sales, Brands, Geography
    - Repeat / Frequency of sales, eg histogram of how many products were bought 1,2,3,4,5 etc times
    - Are low sales linked to anything such as state, age of customer etc
    - anything in the bought with, eg more sales this way?
    - Sales by brand, any patterns 

- Prices
    - stock price vs sales price comparison
    - how many are discounted vs not
    - which are most discounted, does this equate to higher sales etc

- Reviews
    - any messages from the reviews?
    - any interpretation possible?
    - links to brand?
    - anything else like age, geography ...
    - ?? sales probably not linked to reviews, unless I suppose reviews are submitted at a later date
    - ?? follow on sales for a customer?
    - wordcloud from reviews



